In [17]:
import pandas as pd
import numpy as np

# read files
roads = pd.read_csv("_roads3.csv")
bridges = pd.read_excel("BMMS_overview.xlsx")

# select roads columns
roads = roads[["road", "lrp", "lat", "lon", "chainage"]]
roads = roads[roads["road"] == "N1"].copy()

# select bridge columns
bridges = bridges[bridges["road"] == "N1"][["LRPName", "length"]].copy()

# merge
roads = roads.merge(
    bridges,
    how="left",
    left_on="lrp",
    right_on="LRPName"
)

# model_type column
roads["model_type"] = np.select(
    [
        roads["lrp"] == "LRPS",
        roads["lrp"] == "LRPE",
        roads["length"].notna()
    ],
    [
        "source",
        "sink",
        "bridge"
    ],
    default="link"
)

# calculate length based on chainage
chainage_diff = (roads["chainage"].shift(-1) - roads["chainage"])* 1000
roads["length"] = roads["length"].fillna(chainage_diff)

# rename lrp to name
roads = roads.rename(columns={"lrp": "name"})

# drop helper column
roads = roads.drop(columns=["LRPName"])
roads = roads.drop(columns=["chainage"])


# reset index and create proper ID
roads = roads.reset_index(drop=True)
roads.insert(0, "id", roads.index + 1)

roads.to_csv("processed_data.csv", index=False)

display(roads.head(30))


,id,road,name,lat,lon,length,model_type
0,1,N1,LRPS,23.706028,90.443333,814.00,source
1,2,N1,LRPSa,23.702917,90.450417,8.00,link
2,3,N1,LRPSb,23.702778,90.450472,178.00,link
3,4,N1,LRP001,23.702139,90.451972,1000.00,link
4,5,N1,LRP002,23.697889,90.460583,130.00,link
5,6,N1,LRP002a,23.697361,90.461667,870.00,link
6,7,N1,LRP003,23.693833,90.469138,1000.00,link
7,8,N1,LRP004,23.693611,90.478777,175.00,link
8,9,N1,LRP004a,23.693805,90.480527,825.00,link
9,10,N1,LRP005,23.694750,90.488500,1000.00,link
